<a href="https://colab.research.google.com/github/matchbou/Tensorflow20-20190314/blob/master/TF2.0%20Functions%20and%20Autograph-Macrun-Bugs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## その他のリソース

Python、Jupyter、Colaboratory、関連するツールを最大限に活用する方法については、以下のリソースをご覧ください。

### 機械学習集中講座
Google の機械学習に関するオンライン コースからいくつかのノートブックをご紹介します。詳しくは、[全コースのウェブサイト](https://developers.google.com/machine-learning/crash-course/)をご覧ください。
- [Intro to Pandas](/notebooks/mlcc/intro_to_pandas.ipynb)
- [Tensorflow concepts](/notebooks/mlcc/tensorflow_programming_concepts.ipynb)
- [First steps with TensorFlow](/notebooks/mlcc/first_steps_with_tensor_flow.ipynb)
- [Intro to neural nets](/notebooks/mlcc/intro_to_neural_nets.ipynb)
- [Intro to sparse data and embeddings](/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb)

### 機械学習の例: Seedbank

Colaboratory で実現したインタラクティブな機械学習解析のさまざまな例を確認するには、[Seedbank](https://research.google.com/seedbank/)プロジェクトをご覧ください。

おすすめの例をご紹介します。

- [Neural Style Transfer](https://research.google.com/seedbank/seed/neural_style_transfer_with_tfkeras): ディープ ラーニングを使用して画像間でスタイルを転送します。
- [EZ NSynth](https://research.google.com/seedbank/seed/ez_nsynth): 音声と WaveNet オート エンコーダを合成します。
- [Fashion MNIST with Keras and TPUs](https://research.google.com/seedbank/seed/fashion_mnist_with_keras_and_tpus): ファッション関連の画像をディープ ラーニングで分類します。
- [DeepDream](https://research.google.com/seedbank/seed/deepdream): 独自の写真から DeepDream 画像を生成します。
- [Convolutional VAE](https://research.google.com/seedbank/seed/convolutional_vae): 手書きの数字の生成モデルを作成します。

In [1]:
!pip install tf-nightly-gpu-2.0-preview

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [0]:
import tensorflow as tf

In [4]:
@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.31627607, 0.53785014, 0.34887263],
       [0.5049153 , 1.0454628 , 0.35430205],
       [0.21411033, 0.3337625 , 0.2872383 ]], dtype=float32)>

In [5]:
simple_nn_layer

In [6]:
def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))


<tf.Tensor: id=36, shape=(3,), dtype=int32, numpy=array([3, 5, 7], dtype=int32)>

In [7]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


In [8]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=144, shape=(), dtype=int32, numpy=42>

In [9]:
print(tf.autograph.to_code(sum_even.python_function, experimental_optional_features=None))

def tf__sum_even(items):
  do_return = False
  retval_ = None
  s = 0

  def loop_body(loop_vars, s_2):
    c = loop_vars
    continue_ = False
    cond = c % 2 > 0

    def get_state():
      return ()

    def set_state(_):
      pass

    def if_true():
      continue_ = True
      return continue_

    def if_false():
      return continue_
    continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state)
    cond_1 = ag__.not_(continue_)

    def get_state_1():
      return ()

    def set_state_1(_):
      pass

    def if_true_1():
      s_1, = s_2,
      s_1 += c
      return s_1

    def if_false_1():
      return s_2
    s_2 = ag__.if_stmt(cond_1, if_true_1, if_false_1, get_state_1, set_state_1)
    return s_2,
  s, = ag__.for_stmt(items, None, loop_body, (s,))
  do_return = True
  retval_ = s
  return retval_



In [10]:
@tf.function
def fizzbuzz(n):
  msg = tf.constant('')
  for i in tf.range(n):
    if tf.equal(i % 3, 0):
      msg += 'Fizz'
    elif tf.equal(i % 5, 0):
      msg += 'Buzz'
    else:
      msg += tf.as_string(i)
    msg += '\n'
  return msg


print(fizzbuzz(tf.constant(15)).numpy().decode())

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14



In [11]:
class CustomModel(tf.keras.models.Model):
  
  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

<tf.Tensor: id=274, shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>

In [12]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if tf.equal(v % 2, 0):
    v.assign(v + 1)


find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

In [0]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

In [0]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()


In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if tf.equal(step % 10, 0):
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.84759748 ; accuracy 0.286
Step 20 : loss 1.28982151 ; accuracy 0.4805
Step 30 : loss 0.846976876 ; accuracy 0.571
Step 40 : loss 0.692483664 ; accuracy 0.6325
Step 50 : loss 0.628998041 ; accuracy 0.6778
Step 60 : loss 0.500465453 ; accuracy 0.706833363
Step 70 : loss 0.292859852 ; accuracy 0.731285691
Step 80 : loss 0.405897975 ; accuracy 0.748875
Step 90 : loss 0.456417859 ; accuracy 0.765555561
Step 100 : loss 0.336770415 ; accuracy 0.7797
Step 110 : loss 0.221801817 ; accuracy 0.790909111
Step 120 : loss 0.246494755 ; accuracy 0.800416648
Step 130 : loss 0.275208026 ; accuracy 0.808692336
Step 140 : loss 0.195720941 ; accuracy 0.816
Step 150 : loss 0.323756605 ; accuracy 0.822333336
Step 160 : loss 0.290231586 ; accuracy 0.828562498
Step 170 : loss 0.208221838 ; accuracy 0.83441174
Step 180 : loss 0.384253919 ; accuracy 0.839333355


In [0]:
def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x]


square_if_positive(range(-5, 5))